In [1]:
import re
import os
from typing import Literal, Optional, List
from concurrent.futures import ThreadPoolExecutor, as_completed

import dspy
from tqdm import tqdm
from pydantic import BaseModel, Field
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import Filter, Writer, DocInfo, run_chains_with_extarction_history_multi_threads

e:\anaconda\envs\sisyphus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PAPER_DB = 'heas_batch_1'
K = 3
QUERY_SYN = """Experimental procedures describing the synthesis and processing of HEAs materials, including methods such as melting, casting, rolling, annealing, heat treatment, or other fabrication techniques. Details often include specific temperatures (e.g., °C), durations (e.g., hours, minutes), atmospheric conditions (e.g., argon, vacuum), mechanical deformation (e.g., rolling reduction)."""
QUERY_MECHANICAL = "The stress-strain curve of alloy, describes yield strength (ys), tensile strength (uts) and elogation properties, for example, CoCuFeMnNi shows tensile strength of 1300 MPa and total elongation of 20%"
QUERY_PHASE = """Microstructure characterization of alloys (common phases include FCC, BCC, HCP, L12, B2 etc.), usually througth technique like XRD or TEM. Describe about phase and grain size and bouondaries"""
SYSTEM_MESSAGE = """You are an expert at structured data extraction from HEAs (high entropy alloys) domain. You will be given unstructured text from a research paper and should convert it into the given structure"""
INSTRUCTION =  """Extract HEAs tensile and compressive properties from text.
Note:
For HEAs composition
- HEAs composition should be strictly comply with standard naming convention, e.g CoCrFeMnNi. If the alloy is doped or added with other element and author did not provide a nominal composition, you should give name such as W-CoCrFeMnNi. Acronym such as HEA-1 is prohibited.
For mechanical value
- If the value are provided as average +/- standard deviation, use the average value, e.g 700 +/- 30 should be recorded as 700, if the value are provied as minimum-maximum, use the minimum value.
- Prioritize the value in the table over text if there is a conflict.
For synthesis
- As-cast (remelting serval times and casting) materials should not contains thermal mechanical processes.
- Correctly associate synthesis parameters with the corresponding mechanical properties.
"""

In [3]:
class AlloyRecord(BaseModel):
    composition: str = Field(description='The nominal chemical composition of the alloy, ensure the validity of the formula, e.g. CoCrFeNi.')
    phase: Optional[str] = Field(description='The phase of the alloy, such as FCC, BCC, HCP, L12, B2, Laves etc. If there are multiple phases, separate them with commas')
    ys: Optional[float] = Field(description='the value of yield strength, convert to MPa if the unit is not MPa, e.g. 1 GPa -> 1000 MPa')
    uts: Optional[float] = Field(description='the value of ultimate tensile strength (for tensile tests) or maximum compression strength (for compression tests), convert to MPa if the unit is not MPa, e.g. 1GPa -> 1000 MPa')
    strain: Optional[float] = Field(description='the value of plastic elongation or plastic compression strain, convert to percentage if the unit is not percentage, e.g. 1%')
    fabrication: Optional[str] = Field(description='The fabrication method of the alloy, e.g. vacuum arc-melting')
    thermal_mechanical_processings: Optional[str]  = Field(description='The sequential post-processing steps of the alloy separated by vertical bar "|", be briefly, eg., annealed at 900 °C for 4 h | homogenized at 1200 °C for 2 h')

    test_type: Literal['tensile', 'compressive']
    test_temperature: str = Field(description='The temperature at which the mechanical properties were tested, e.g. 25 °C. If the temperature is given in Kelvin, convert it to Celsius by subtracting 273. If not mentioned in the text, record it as 25 °C.')

class Records(BaseModel):
    records: Optional[List[AlloyRecord]] = Field(description='The records of the alloy properties')


lm = dspy.LM('openai/gpt-4o-mini', cache=False, temperature=0)
dspy.configure(lm=lm)
model = ChatOpenAI(model='gpt-4o', temperature=0.0)
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(collection_name='hea_collection', embedding_function=embeddings)
article_db = get_plain_articledb(PAPER_DB)
extract_template = ChatPromptTemplate(
    [
        ('system', SYSTEM_MESSAGE),
        ('user', '[START OF PAPER]{paper}[END OF PAPER]\n\nInstruction:\n{instruction}')
    ]
)
distill_extract_chain = extract_template | model.with_structured_output(Records, method='json_schema')

def retrieve(vector_store, source, query, sub_titles, k):
    if not sub_titles:
        filter_ = {"source": source}
    else:
        filter_ = {"$and":[{
            "sub_titles": {
                "$in": sub_titles
            }},
            {"source": source
        }]}
    return vector_store.similarity_search(
        query,
        k=k,
        filter=filter_
    )

def match_subtitles(docs, pattern):
    sub_titles = list(set([doc.metadata["sub_titles"] for doc in docs]))
    target_titles = []
    for title in sub_titles:
        if pattern.search(title):
            target_titles.append(title)
    return target_titles

def get_target_para_lm(source, sub_titles, query, classifier, class_, k=K, vector_store=vector_store):
    candidates = retrieve(vector_store, source, query, sub_titles, k)
    final = []
    with ThreadPoolExecutor(5) as worker:
        futures = [worker.submit(classifier, paragraph=candidate.page_content) for candidate in candidates]
        future_doc = dict(zip(futures, candidates))
        for future in as_completed(futures):
            if future.result().topic == class_:
                final.append(future_doc[future])
    return final

def get_target_para_regex(source, sub_titles, query, regex_pattern, k=K, vector_store=vector_store):
    candidates = retrieve(vector_store, source, query, sub_titles, k)
    final = [c for c in candidates if re.search(regex_pattern, c.page_content)]
    return final

def get_tables(docs):
    tables = [doc for doc in docs if doc.metadata["sub_titles"] == "table"]
    final = []
    with ThreadPoolExecutor(5) as worker:
        futures = [worker.submit(classifier_table, paragraph=table.page_content) for table in tables]
        future_doc = dict(zip(futures, tables))
        for future in as_completed(futures):
            pred = future.result()
            if pred.contain and pred.label == 'experimental':
                final.append(future_doc[future])
    return final

class ClassifySyn(dspy.Signature):
    """assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include synthesis, characterization, and others.
    Note: a qualified synthesis paragraph should include the synthesis and processing of materials, including methods such as melting, casting, rolling, annealing, heat treatment.be very strict about your decision."""
    paragraph: str = dspy.InputField()
    topic: Literal['synthesis', 'characterization', 'others'] = dspy.OutputField()

classifier_syn = dspy.ChainOfThought(signature=ClassifySyn)

class ClassifyTable(dspy.Signature):
    """assign labels to tables inside HEAs(high entropy alloys) papers. First classify the table to experimental or theoretical, then inspecting the table to find whether it contains properties relevant to tensile or compressive."""
    paragraph: str = dspy.InputField()
    label: Literal['experimental', 'theoretical'] = dspy.OutputField(desc="whether the contents of table are experimental measured or theoretical calculated.")
    contain: bool = dspy.OutputField(desc='whether the table includes any of these values: yield strength, tensile/compressive strength, or strain of HEAs')

classifier_table = dspy.ChainOfThought(signature=ClassifyTable)

class ClassifyPaper(dspy.Signature):
    """assign label to HEAs (high entropy alloys) paper based on their title and abstract."""
    title: str = dspy.InputField()
    abstract: str = dspy.InputField()
    label: Literal['hea_experimental', 'hea_theoretical', 'irrelevant'] = dspy.OutputField(desc="Pay attention to keywords such as 'molecular dynamics' or 'machine learning,' which should be labeled as hea_theoretical. Label keywords related to fabrication processes as hea_experimental.")
    mechanical_relevancy: bool = dspy.OutputField(desc='whether this paper describe the mechanical properties such as tensile or compressive')

classifier_paper = dspy.ChainOfThought(signature=ClassifyPaper)

def get_meta_info(docs):
    metadata = docs[0].metadata
    source = metadata['source']
    title = metadata['title']
    doi = metadata['doi']
    return source, title, doi

def reorder_docs(ordered, docs):
    """reorder the retrieved documents.
    Note: this function can deal with duplication in the docs!
    WARNING: do not modify the docs"""
    with_order = []
    used_i = []
    for doc in docs:
        for o_doc, i in ordered:
            if i in used_i:
                continue
            if doc == o_doc:
                with_order.append((doc, i))
                used_i.append(i)
                break
    final = sorted(with_order, key=lambda x: x[1])
    return [el[0] for el in final]

def render_docs(docs, title):
    """render docs to nicely formatted paper look.
    Since the tables are the most information dense format, we put it at the tail"""
    tables = [doc for doc in docs if doc.metadata['sub_titles'] == 'table']
    paras = [doc for doc in docs if doc.metadata['sub_titles'] != 'table']

    previous_titles = []
    scratch_pad = [title]
    for para in paras:
        sub_titles = para.metadata['sub_titles'].split('/')
        title_to_write = [title for title in sub_titles if title not in previous_titles] 
        previous_titles = sub_titles
        rendered_text = '\n'.join(title_to_write + [para.page_content])
        if title_to_write:
            rendered_text = '\n' + rendered_text
        scratch_pad.append(rendered_text)

    if tables:
        scratch_pad.append('\nMechanical property tables')
    for table in tables:
        scratch_pad.append('\n' + table.page_content)

    return '\n'.join(scratch_pad)

In [4]:
def distill_extract(docs):
    with_order = list(zip(docs, range(len(docs))))
    source, title, doi = get_meta_info(docs)
    abstract_docs = [doc for doc in docs if doc.metadata['sub_titles'] == 'Abstract']
    last_intro_doc = [doc for doc in docs if re.search(r'introduction', doc.metadata['sub_titles'], re.I)][-1:] # last para of intro often contains key information

    syn_pattern = re.compile(r'(experiment)|(preparation)|(method)', re.I)
    res_pattern = re.compile(r'result', re.I)
    mech_pattern = re.compile(r'(\b(MPa|GPa)\b|\d+(\.\d)?\s*%)')
    phase_pattern = re.compile(r'\b(FCC|BCC|HCP|L12|B2|Laves|f.c.c.|b.c.c.|h.c.p.|face-centered cubic|body-centered cubic|hexagonal close-packed)\b', re.I)
    sub_titles_syn = match_subtitles(docs, syn_pattern)
    sub_titles_res = match_subtitles(docs, res_pattern)

    # classify paper
    pred = classifier_paper(title=title, abstract='\n'.join(doc.page_content for doc in abstract_docs))
    if not (pred.mechanical_relevancy and pred.label == 'hea_experimental'):
        return

    # when there is only abstract available
    if len(docs) == 1:
        if mech_pattern.search(docs[0].page_content):
            return (docs[0], distill_extract_chain.invoke({'paper': render_docs(docs, title=title), 'instruction': INSTRUCTION}).records)
        return
    
    # embedding docs
    vector_store.add_documents(docs)
    
    # retain relevant paragraphs then extraction
    tensile_docs = get_target_para_regex(source, sub_titles_res, QUERY_MECHANICAL, mech_pattern, k=5)
    tensile_tables = get_tables(docs)
    if not (tensile_docs or tensile_tables):
        return # no tensile relevant text/tables found
    synthesis_docs = get_target_para_lm(source, sub_titles_syn, QUERY_SYN, classifier_syn, 'synthesis')
    phase_docs = get_target_para_regex(source, sub_titles_res, QUERY_PHASE, phase_pattern)
    disordered_docs = tensile_docs + tensile_tables + synthesis_docs + phase_docs + abstract_docs + last_intro_doc
    ordered_docs = reorder_docs(with_order, disordered_docs)
    paper = render_docs(ordered_docs, title)

    results = distill_extract_chain.invoke({'paper': paper, 'instruction': INSTRUCTION})
    doc = Document(page_content=paper, metadata={'source': source, 'doi': doi})
    return doc, results.records

def return_valid(t):
    if t is None:
        return
    doc, result = t
    if result:
        return [DocInfo(doc, result)]
    return

In [7]:
docs_getter = Filter(article_db)
result_db = get_create_resultdb('heas_20_distill', AlloyRecord)
chain = docs_getter + distill_extract + return_valid + Writer(result_db)
chain_test = docs_getter + distill_extract + return_valid 

e:\anaconda\envs\sisyphus\lib\site-packages\pydantic\main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [6]:
test_articles = os.listdir('articles_processed')[:20]
test_articles

['10.1002&sol;adem.201900587.html',
 '10.1002&sol;adem.202200523.html',
 '10.1002&sol;adma.201807142.html',
 '10.1002&sol;mawe.202300263.html',
 '10.1016&sol;j.actamat.2020.08.043.html',
 '10.1016&sol;j.actamat.2021.117571.html',
 '10.1016&sol;j.actamat.2022.118516.html',
 '10.1016&sol;j.actamat.2024.120498.html',
 '10.1016&sol;j.actamat.2024.120614.html',
 '10.1016&sol;j.addma.2020.101202.html',
 '10.1016&sol;j.commatsci.2022.111888.html',
 '10.1016&sol;j.corsci.2021.110073.html',
 '10.1016&sol;j.corsci.2022.110357.html',
 '10.1016&sol;j.corsci.2023.111422.html',
 '10.1016&sol;j.corsci.2023.111708.html',
 '10.1016&sol;j.ijhydene.2019.04.280.html',
 '10.1016&sol;j.ijhydene.2020.11.154.html',
 '10.1016&sol;j.ijrmhm.2023.106163.html',
 '10.1016&sol;j.ijrmhm.2023.106225.html',
 '10.1016&sol;j.intermet.2013.03.018.html']

In [8]:
run_chains_with_extarction_history_multi_threads(chain, None, 5,'heas_20_distill', None, test_articles)

100%|██████████| 20/20 [01:16<00:00,  3.81s/it]


In [12]:
import json
with open('heas_20_distill.json', 'w', encoding='utf8') as f:
    json.dump(result_db.load_as_json('4o-mini & 4o', instruction=SYSTEM_MESSAGE + '\n' +  INSTRUCTION, db_name='heas_20_distill', with_doi=True), f, indent=2, ensure_ascii=False)

In [10]:
results = chain_test.compose(test_articles[0])

[DocInfo(doc=Document(metadata={'source': '10.1002&sol;adem.201900587.html', 'doi': '10.1002/adem.201900587'}, page_content='Effect of Initial Grain Size on Deformation Mechanism during High‐Pressure Torsion in V10Cr15Mn5Fe35Co10Ni25 High‐Entropy Alloy\n\nAbstract\nThe transition of the deformation mechanism from the dislocation slip-mediated mechanism to the twin-mediated mechanism with increasing grain size is a well-observed phenomenon in materials with low stacking fault energy during compression/tensile tests. To understand this effect further at large strains, a V10Cr15Mn5Fe35Co10Ni25 (at%) high-entropy alloy with two initial average grain sizes is processed by high-pressure torsion (HPT) at different numbers of turns. The results indicate that initial grain size plays a significant role in the deformation mechanism during the HPT process. The fine-grained (FG) sample exhibits only a tangled dislocation structure, whereas mechanical twins are observed along with the formation of 

In [12]:
results[0].info

[AlloyRecord(composition='CoCrFeMnNiV', phase='FCC', ys=430.0, uts=720.0, strain=48.1, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min', test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='CoCrFeMnNiV', phase='FCC', ys=230.0, uts=532.0, strain=57.6, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 1100 °C for 60 min', test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='CoCrFeMnNiV', phase='FCC', ys=1120.0, uts=1447.0, strain=15.9, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min | HPT at 6 GPa, 1/4 turn', test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='CoCrFeMnNiV', phase='FCC', ys=1

In [13]:
chain_test.compose('10.1016&sol;j.ijrmhm.2023.106163.html')[0].info

[AlloyRecord(composition='Nb3Ta3(Ti2Ni)4', phase='BCC', ys=395.0, uts=None, strain=50.0, fabrication='Laser powder bed melting', thermal_mechanical_processings=None, test_type='compressive', test_temperature='25 °C'),
 AlloyRecord(composition='Nb3Ta3Mo(Ti2Ni)3', phase='BCC, TiNi, FCC', ys=915.0, uts=None, strain=50.0, fabrication='Laser powder bed melting', thermal_mechanical_processings=None, test_type='compressive', test_temperature='25 °C'),
 AlloyRecord(composition='Nb3Ta3Mo2(Ti2Ni)2', phase='BCC', ys=1285.0, uts=2447.0, strain=27.1, fabrication='Laser powder bed melting', thermal_mechanical_processings=None, test_type='compressive', test_temperature='25 °C'),
 AlloyRecord(composition='Nb3Ta3(Ti2Ni)4', phase='BCC', ys=671.0, uts=1036.0, strain=9.2, fabrication='Laser powder bed melting', thermal_mechanical_processings=None, test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='Nb3Ta3Mo(Ti2Ni)3', phase='BCC, TiNi, FCC', ys=1184.0, uts=1403.0, strain=4.4, fabricat

In [16]:
chain_test.compose('10.1016&sol;j.ijrmhm.2023.106225.html')[0].info

[AlloyRecord(composition='W-Al0.5Cr0.9FeNi2.5V0.2', phase='FCC, L12', ys=880.0, uts=1222.0, strain=14.2, fabrication='3D printing', thermal_mechanical_processings=None, test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='W-Al0.5Cr0.9FeNi2.5V0.2', phase='FCC, L12', ys=921.0, uts=1290.0, strain=11.8, fabrication='3D printing', thermal_mechanical_processings='aged at 700 °C for 4 h', test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='W-Al0.5Cr0.9FeNi2.5V0.2', phase='FCC, L12', ys=939.0, uts=1244.0, strain=3.9, fabrication='3D printing', thermal_mechanical_processings='aged at 700 °C for 16 h', test_type='tensile', test_temperature='25 °C'),
 AlloyRecord(composition='W-Al0.5Cr0.9FeNi2.5V0.2', phase='FCC, L12', ys=None, uts=2100.0, strain=None, fabrication='3D printing', thermal_mechanical_processings=None, test_type='compressive', test_temperature='25 °C')]

In [ ]:
# issues
# lose information which was not synthesis in this article which was used for comparison.